## ROI signal extraction

This script performs extraction and cleaning of BOLD signal from predefined brain regions. These signals are used to perform PPI GLM modeling. Two versions of signals are prepared – **raw** (without confound removal) and **clean** (with confounds removed). Raw version is used for deconvolution implemented in `spm_peb_ppi.m` whereas clean version is used for custom deconvolution using Ridge regression. For both versions signal is:
- standardized using z-score
- detrended
- high pass filtered with 128s cut-off

Then signal is extracted from all predefined ROIs. For clean version, 24 head motion parameters (hmp) are included as confounds, raw version omit regressing out any confounds. Output data is stored in two aggregate arrays with signals for all subjects, conditions and rois (both have shape $N_{subjects} \times N_{conditions} \times N_{volumes} \times N_{rois}$):
- `time_series_clean_all`: signals with hmp variability removed
- `time_series_raw_all`: signal with no confoud removal

---
**Last update**: 28.04.2020 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tempfile

from bids import BIDSLayout
from nilearn.input_data import NiftiSpheresMasker
from scipy import io

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data                    

In [ ]:
# Directory for PPI analysis
path_out = os.path.join(path_root, 
                        'data/main_fmri_study/derivatives/ppi')
os.makedirs(path_out, exist_ok=True)

path_parcellations = os.path.join(path_out, 'parcellations')
path_timeries = os.path.join(path_out, 'timeseries')

# Atlases
path_300_roi = os.path.join(path_parcellations, '300_ROI_Set/ROIs_300inVol_MNI_allInfo.txt')

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path=path_beh)
n_subjects, n_conditions, n_trials, _ = beh.shape
n_volumes = 730

### Query neuroimaging dataset

Using BIDSLayout object query BIDS dataset to pull out necessary files.
- `fmri_files`: list of two lists containing sorted (by subject number) paths to imaging files, first list corresponds to reward condition of PRL task and second list corresponds to punishment condition of PRL task
- `conf_files`: list of two lists containing sorted (by subject number) paths to confound files

In [ ]:
path_bids = os.path.join(path_root, 'data/main_fmri_study')

layout = BIDSLayout(
    root=path_bids,
    derivatives=True,
    validate=True,
    index_metadata=False
)

fmri_filter = {
    "extension": [".nii", ".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "bold",
    "desc": "preproc",
    "return_type": "filename"
}

conf_filter = {
    "extension": "tsv",
    "desc": "confounds",
    "return_type": "filename"
}

fmri_files, conf_files = [], []

for task_dict in [{"task": "prlrew"}, {"task": "prlpun"}]:
    fmri_filter.update(task_dict)
    conf_filter.update(task_dict)
    fmri_files.append(layout.get(**fmri_filter))
    conf_files.append(layout.get(**conf_filter))

### Load brain parcellation

Here brain parcellation is loaded. Parcellaton should be stored and formatted as csv file with (at least these) columns:

| Column Name | Description |
|:--:|----|
| `x` | x MNI coordinate for ROI center | 
| `y` | y MNI coordinate for ROI center | 
| `z` | z MNI coordinate for ROI center | 
| `radius(mm)` | sphere radius in mm |

Since `NiftiSpheresMasker` class accepts only single radius value, different maskers are created for all unique radii values (if needed). 

In [ ]:
path_atlas = path_300_roi

df_atlas = pd.read_csv(path_300_roi, sep=' ')
seeds = [tuple(coords[1]) for coords in df_atlas[['x', 'y', 'z']].iterrows()]
n_rois = len(seeds)

# Dict for storing maskers for different radii values
rois = {}

for radius in df_atlas['radius(mm)'].unique():
    # Select ROIs with given radius
    roi_indices = np.flatnonzero(df_atlas['radius(mm)'] == radius)
    
    # Create masker for single radius value
    masker = NiftiSpheresMasker(
        [seeds[idx] for idx in roi_indices], 
        radius=radius,                
        mask_img=None,            
        allow_overlap=True, 
        standardize='zscore', 
        detrend=True, 
        high_pass=1/128,
        t_r=2
    )
    
    rois[radius] = {'masker': masker, 'indices': roi_indices}

### Extract and clean signal

Resulting timecourses are stored in 4-dimensional array with dimensions corresponding to subjects, tasks, volumes and roi's. Matrix shape is `n_subjects` $\times$ `n_conditions` $\times$ `n_volumes` $\times$ `n_rois`. 

In [ ]:
time_series_clean_all = np.zeros((n_subjects, n_conditions, n_volumes, n_rois))
time_series_raw_all = np.zeros((n_subjects, n_conditions, n_volumes, n_rois))
confounds_hmp_all = np.zeros((n_subjects, n_conditions, n_volumes, 24))

for task_idx in range(n_conditions):
    for sub_idx in range(n_subjects):
        
        fmri_fname = fmri_files[task_idx][sub_idx]
        conf_fname = conf_files[task_idx][sub_idx]

        # Read confounds, filtering & save confounds
        conf_df = pd.read_csv(conf_fname, sep='\t')
        conf_df = conf_df.filter(regex='rot|trans')
        conf_df = conf_df.fillna(0)

        tmp_fname = 'temp'
        conf_df.to_csv(tmp_fname)

        time_series_clean = np.zeros((n_volumes, n_rois))
        time_series_raw = np.zeros((n_volumes, n_rois))
        for radius in rois:
            # Get indices to insert computed timeseries into right positions
            roi_indices = rois[radius]['indices']

            # Extract timeseries
            time_series_clean[:, roi_indices] = rois[radius]['masker'].fit_transform(
                fmri_fname,
                confounds=tmp_fname
            )
            time_series_raw[:, roi_indices] = rois[radius]['masker'].fit_transform(
                fmri_fname,
                confounds=None
            )
            
        # Store results in corresponding arrays
        time_series_clean_all[sub_idx][task_idx] = time_series_clean
        time_series_raw_all[sub_idx][task_idx] = time_series_raw
        confounds_hmp_all[sub_idx][task_idx] = conf_df
        
        os.remove(tmp_fname)
    
# Save in numpy format
np.save(os.path.join(path_timeries, 'time_series_clean_all'), time_series_clean_all)

# Save for further SPM use
io.savemat(os.path.join(path_timeries, 'time_series_raw_all.mat'), 
           {'time_series_raw_all': time_series_raw_all})
io.savemat(os.path.join(path_timeries, 'confounds_hmp_all.mat'),
           {'confounds_hmp_all': confounds_hmp_all})

### Plot extracted signal from single subject

Show carpet plot of cleaned BOLD signal from all predefined ROIs.

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10), facecolor='w')

carpet = ax.imshow(
    time_series_clean_all[0][0].T, 
    aspect='auto',
    cmap='gray'
)

ax.set_ylabel('ROI')
ax.set_xlabel('Time [s]')
ax.set_xticklabels([f'{t:.0f}' for t in 2 * ax.get_xticks()])
ax.set_title('ROI Timecourse Carpetplot')

fig.colorbar(carpet, pad=0.01, shrink=1, aspect=10)
plt.show()